# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

### Task
- Ceate a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data
- Focus on book ratings and genres
- The script should filter books with ratings above a certain threshold and in specific genres. 
- Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)

Goal:
- Create script with Python, Beautifulsoup and pandas
- Scrape Book Data
- Focus on book ratings and genres
- Filter books by rating (above certain threshold)
- Filter books by genre
- Structure scraped data in a tabular format using pandas

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_books(min_rating=4.0, max_price=20.0):
    base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
    books_data = []
    
    for page in range(1, 6): #example range, can be adjusted
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        books = soup.find_all('article', class_='product_pod') #find book containers on page
        
        for book in books:
            title = book.h3.a['title']

            # price
            price_str = book.find('p', class_='price_color').text
            price = float(re.sub(r'[^\d.]+', '', price_str))  # remove all non-numeric characters
            
            # rating
            rating_class = book.find('p', class_='star-rating')['class'][1]
            rating_map = {
                'One': 1,
                'Two': 2,
                'Three': 3,
                'Four': 4,
                'Five': 5
            }
            rating = rating_map[rating_class]

            # availability
            availability = book.find('p', class_='instock availability').text.strip()

            # book details
            detail_page_url = book.h3.a['href']
            detail_response = requests.get(f'http://books.toscrape.com/catalogue/{detail_page_url}')
            detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
            
            # genre
            genre_element = detail_soup.find('a', href=lambda x: x and 'catalogue/category/books' in x)
            genre = genre_element.text if genre_element else 'Unknown'
            
            # description
            description_meta = detail_soup.find('meta', attrs={'name': 'description'})
            description = description_meta['content'].strip() if description_meta else 'No Description'  # Default if not found
            
            # filter by rating and price
            if rating >= min_rating and price <= max_price:
                books_data.append({
                    'UPC': '',  # UPC is not available on the main page
                    'Title': title,
                    'Price (£)': price,
                    'Rating': rating,
                    'Genre': genre,
                    'Availability': availability,
                    'Description': description
                })
    
    # create df from the scraped data
    df = pd.DataFrame(books_data)
    return df

# example
if __name__ == "__main__":
    result_df = scrape_books(min_rating=4.0, max_price=20.0)
    print(result_df)


result_df.head(25)

  UPC                                              Title  Price (£)  Rating  \
0                                            Set Me Free      17.46       5   
1      The Four Agreements: A Practical Guide to Pers...      17.66       5   
2                                         Sophie's World      15.94       5   
3                Untitled Collection: Sabbath Poems 2014      14.27       4   
4                                        This One Summer      19.49       4   
5                                                 Thirst      17.27       5   
6      Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...      13.61       5   
7       Princess Between Worlds (Wide-Awake Princess #5)      13.34       5   
8      Outcast, Vol. 1: A Darkness Surrounds Him (Out...      15.44       4   
9      Mama Tried: Traditional Italian Cooking for th...      14.02       4   

           Genre Availability  \
0  Unknown Genre     In stock   
1  Unknown Genre     In stock   
2  Unknown Genre     In stock  

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,,Set Me Free,17.46,5,Unknown Genre,In stock,Aaron Ledbetterâs future had been planned ou...
1,,The Four Agreements: A Practical Guide to Pers...,17.66,5,Unknown Genre,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,,Sophie's World,15.94,5,Unknown Genre,In stock,A page-turning novel that is also an explorati...
3,,Untitled Collection: Sabbath Poems 2014,14.27,4,Unknown Genre,In stock,"More than thirty-five years ago, when the weat..."
4,,This One Summer,19.49,4,Unknown Genre,In stock,"Every summer, Rose goes with her mom and dad t..."
5,,Thirst,17.27,5,Unknown Genre,In stock,"On a searing summer Friday, Eddie Chapman has ..."
6,,"Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Pr...",13.61,5,Unknown Genre,In stock,THE LONG-AWAITED STORY OF FANGIRLS TAKING ON T...
7,,Princess Between Worlds (Wide-Awake Princess #5),13.34,5,Unknown Genre,In stock,Just as Annie and Liam are busy making plans t...
8,,"Outcast, Vol. 1: A Darkness Surrounds Him (Out...",15.44,4,Unknown Genre,In stock,NEW HORROR SERIES FROM THE WALKING DEAD CREATO...
9,,Mama Tried: Traditional Italian Cooking for th...,14.02,4,Unknown Genre,In stock,Cecilia Granata grew up cooking with her famil...


**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).

Execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

### Expected Outcome

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:


### Expected Outcome

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).

In [16]:
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

SyntaxError: invalid character '£' (U+00A3) (3436091407.py, line 1)